# Multi class logisitic regression

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Exploratory Data Analysis and Visualization

In [ ]:
df = pd.read_csv('iris.csv')
df.head()

In [ ]:
df['species'].value_counts()

In [ ]:
sns.scatterplot(data=df, x='sepal_length', y='sepal_width', hue='species')

In [ ]:
sns.scatterplot(data=df, x='petal_length', y='petal_width', hue='species')

In [ ]:
sns.pairplot(df, hue='species')

In [ ]:
sns.heatmap(df.corr(numeric_only=True), annot=True)

## Train and Test Splits

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('species', axis=1)
y = df['species']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=99)

## Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

## Creating the model

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [ ]:
log_model = LogisticRegression(multi_class="ovr", solver='saga', max_iter=5000)

In [ ]:
penalty = ['l1', 'l2']
C = np.logspace(0, 4, 10)

grid_model = GridSearchCV(log_model, param_grid={'C': C, 'penalty': penalty })

In [ ]:
grid_model.fit(scaled_X_train, y_train)

In [ ]:
grid_model.best_params_

## Model Performance

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay

In [ ]:
grid_model.best_params_

### Predictions on the test data

In [ ]:
y_pred = grid_model.predict(scaled_X_test)

### Accuracy

In [ ]:
accuracy_score(y_test, y_pred)

### Confusion matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_pred)
cnf_matrix

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cnf_matrix)
disp.plot()

### Classification report

In [ ]:
print(classification_report(y_test, y_pred))

### ROC curve and AUC

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
def plot_multiclass_roc(clf, X_test, y_test, n_classes, figsize=(5, 5)):
    y_score = clf.decision_function(X_test)

    # structures
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    # calculate dummies once
    y_test_dummies = pd.get_dummies(y_test, drop_first=False).values
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_dummies[:, i], y_score[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # roc for each class
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot([0, 1], [0, 1], 'k--')
    ax.set_xlim([0.0, 1.0])
    ax.set_ylim([0.0, 1.05])
    ax.set_xlabel('False Positive Rate')
    ax.set_ylabel('True Positive Rate')
    ax.set_title('Receiver operating characteristic example')
    for i in range(n_classes):
        ax.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f) for label %i' % (roc_auc[i], i))
    ax.legend(loc="best")
    ax.grid(alpha=.4)
    sns.despine()
    plt.show()

In [ ]:
plot_multiclass_roc(grid_model, scaled_X_test, y_test, n_classes=3, figsize=(16, 10))